In [3]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [14]:
from ipynb.fs.full.Oscar_information_3 import nominees_df

In [15]:
nominees_df.to_csv("Oscar_information.csv", index=False) 

In [16]:
import pandas as pd


In [17]:
#Creating a list of urls for 1991 - 1997 Movies
nominees_df_unique = nominees_df[['year', 'movie']].drop_duplicates() #dropping duplicates
movies = nominees_df_unique.values 

#preparing empty lists to store the urls by year
url_91 = []
url_92 = []
url_93 = []
url_94 = []
url_95 = []
url_96 = []
url_97 = []

def titles(movie): #function that replaces the spaces in the movie titles with dashes for webscraping 
    for i in movie:
        return movie.replace(" ", "-") 

#Main loop that builds URLS
for i in movies:
    year, movie_name = i
    url_ind = 'https://www.the-numbers.com/movie/'+titles(movie_name)+'#tab=box-office' #testing url
    if year == 1991:
        url_91.append(url_ind)
    elif year == 1992:
        url_92.append(url_ind)
    elif year == 1993:
        url_93.append(url_ind)
    elif year == 1994:
        url_94.append(url_ind)
    elif year == 1995:
        url_95.append(url_ind)
    elif year == 1996:
        url_96.append(url_ind)
    elif year == 1997:
        url_97.append(url_ind)

In [20]:
#Creating a list of urls for 2015 - 2016 Movies
nominees_df_unique = nominees_df[['year', 'movie']].drop_duplicates()
movies = nominees_df_unique.values 

#preparing empty lists to store the urls by year
url_15 = []
url_16 = []
url_17 = []
url_18 = []
url_19 = []
url_20 = []
url_21 = []

def titles(movie):  #function that replaces the spaces in the movie titles with dashes for webscraping
    for i in movie:
        return movie.replace(" ", "-")

#Main loop that builds URLS
for i in movies:
    year, movie_name = i
    url_ind = 'https://www.the-numbers.com/movie/'+titles(movie_name)+'#tab=box-office' #testing url
    # print(url_ind)
    if year == 2015:
        url_15.append(url_ind)
    elif year == 2016:
        url_16.append(url_ind)
    elif year == 2017:
        url_17.append(url_ind)
    elif year == 2018:
        url_18.append(url_ind)
    elif year == 2019:
        url_19.append(url_ind)
    elif year == 2020:
        url_20.append(url_ind)
    elif year == 2021:
        url_21.append(url_ind)
    # url.append(url_ind)

# Class 

In [23]:
#Entire class made with the aid of ChatGPT

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

class BoxOfficeNumbersScraper:
    def __init__(self, urls, retries=3, delay=3):
        """
        urls: list of movie URLs (e.g., url_91, url_92, etc.)
        retries: number of fetch retries
        delay: delay between retries
        """
        self.urls = list(dict.fromkeys(urls))  # remove duplicates
        self.retries = retries
        self.delay = delay
        self.user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
            "Mozilla/5.0 (X11; Linux x86_64)",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)",
        ]

    def fetch_page(self, url):
        """Fetch page with retries and random user-agent."""
        for attempt in range(self.retries):
            try:
                headers = {"User-Agent": random.choice(self.user_agents)}  # Randomly choose a User-Agent so the server doesn't block you
                resp = requests.get(url, headers=headers, timeout=40) # Try requesting the URL with a 40-second timeout
                resp.raise_for_status()  # raise exception if there is an error
                return BeautifulSoup(resp.text, "html.parser") # If request was successful, parse HTML with BeautifulSoup and return it
            except Exception:  # If this was the FINAL url attempt, re-raise the error so the caller knows it failed
                if attempt == self.retries - 1:
                    raise
                time.sleep(self.delay) # Otherwise, wait a bit before retrying

    
    
    @staticmethod # This method does not take (self). behaves like a regular function
    def extract_domestic_gross(soup):
        """
        Extract the domestic box office gross from a BeautifulSoup object.
        Returns an integer (e.g., 12345678) or None if not found.
        """
        # Look for any element containing 'Domestic Box Office'
        elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))
        for element in elements:
            parent = element.parent
            if parent:
                txt = parent.get_text(" ", strip=True) # Get all text in parent
                num = re.search(r"\$[\d,]+", txt)  # Look for $number pattern
                if num:
                    return int(num.group(0).replace(",", "").replace("$", ""))
        return None



    @staticmethod
    def clean_title_braces(title):
        """Remove {...} from movie title."""
        return re.sub(r"\{.*?\}", "", title).strip()

    @staticmethod
    def canonicalize_title(title):
        """Remove apostrophes, periods, normalize spaces into dashes."""
        transformed_title = title.replace("'", "").replace(".", "")
        transformed_title = re.sub(r"\s+", "-", transformed_title)
        transformed_title = re.sub(r"-{2,}", "-", transformed_title)
        return transformed_title.strip("-")

    @staticmethod
    def rotate_title(title):
        """Rotate first dash-separated segment to the end.
        Example: rotate_title("Back-in-the-Day") --> "in-the-Day-Back" """
        parts = title.split("-")
        if len(parts) > 1:
            return "-".join(parts[1:] + [parts[0]])
        return title


    def try_fallbacks(self, original_title, tried_urls):
        clean = self.canonicalize_title(original_title)
        candidates = [clean]
    
        # Add "&" → "and" replacement
        if "&" in clean or "and" not in clean:
            candidates.append(clean.replace("&", "and"))
    
        # Add rotated versions ("King-Lion" → "Lion-King")
        for cand in list(candidates):
            rotated = self.rotate_title(cand)
            if rotated not in candidates:
                candidates.append(rotated)
    
        base = "https://www.the-numbers.com/movie/{title}#tab=box-office"
    
        # # Try each fallback candidate
        # for t in candidates:
        #     url_try = base.format(title=t)
    
        #     if url_try in tried_urls:
        #         continue
    
        #     tried_urls.add(url_try)

        
    def try_fallbacks(self, original_title, tried_urls):
        """Try multiple title transformations until table found. Return soup, used_url, or None."""
        # Step 1: Clean the original title
        clean = self.canonicalize_title(original_title)
        url_candidates = [clean]
    
        # Step 2: Handle "&" in titles
        # If the title contains "&" or doesn't already have "and", add a version with "&" replaced by "and"
        if "&" in clean or "and" not in clean:
            url_candidates.append(clean.replace("&", "and"))
    
        # Step 3: Generate rotated title variations
        # Example: "Back-in-the-Day" --> "in-the-Day-Back"
        for candidate in list(url_candidates):
            rotated_title = self.rotate_title(candidate) # rotate first dash-separated word to the end
            if rotated_title not in url_candidates:
                url_candidates.append(rotated_title)
                
        # Step 4: Try each candidate to construct a URL
        base_url_fmt = "https://www.the-numbers.com/movie/{title}#tab=box-office"
        for candidate in url_candidates:
            url_try = base_url_fmt.format(title=candidate)
            if url_try in tried_urls:
                continue
            tried_urls.add(url_try)
    
            try:
                soup = self.fetch_page(url_try)
                # SUCCESS → return immediately
                return soup, url_try
            except Exception:
                continue
    
        # All failed
        return None, None


    def scrape_all(self):
        """Main loop to scrape all URLs in self.urls."""
        
        results = []
    
        for i, url in enumerate(self.urls, start=1):  # Loop through each URL with counter (i = index, url = link)
            print(f"\n--- Scraping {i}/{len(self.urls)}: {url} ---")
    
            tried_urls = set() # Keep track of URLs we’ve already attempted
            original_title = url.split("/movie/")[-1].split("#")[0]
    
            # Step 1: Try the original URL
            try:
                soup = self.fetch_page(url) # Fetch the webpage and parse as BeautifulSoup
                tried_urls.add(url) # Mark as tried
            except Exception:
                soup = None
    
            # Step 2: If no table found, try fallback URLs
            if soup is None:
                soup, fallback_url = self.try_fallbacks(original_title, tried_urls)
    
                if soup is None:
                    # Everything failed → append empty result
                    results.append({
                        "title": original_title,
                        "url": url,
                        "domestic_gross": None,
                        "data": None
                    })
                    continue
                else:
                    url = fallback_url  # update to the working fallback URL
    
            # Step 3: No table extraction anymore
            df = None
    
            # Step 4: Extract domestic gros
            raw_domestic = self.extract_domestic_gross(soup)
            domestic = float(raw_domestic) if raw_domestic is not None else None

            # Step 5: Clean the movie title
            cleaned_title = self.clean_title_braces(original_title) # Remove any {...} in title
    
            # Step 6: Save the results for this movie
            results.append({
                "title": cleaned_title,
                "url": url,
                "domestic_gross": domestic,
                "data": df  # always None now
            })
    
        return results


    def summary_domestic(self, results):
        """Convert scraped results into DataFrame with movie, url, and domestic gross box office number."""
        summary = []
        for movie_data in results:
            summary.append({
                "movie": movie_data["title"],
                "url": movie_data["url"],
                "domestic_gross": movie_data["domestic_gross"]
            })
        return pd.DataFrame(summary)


# URL 91

In [24]:
# your URL list


 # Creating instance
scraper = BoxOfficeNumbersScraper(url_91)

# scraping urls
results = scraper.scrape_all()

 # extracting domestic gross from each movie
summary = scraper.summary_domestic(results)

print(summary) # printing data frame
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)  #saving to csv file



--- Scraping 1/14: https://www.the-numbers.com/movie/Bugsy#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/14: https://www.the-numbers.com/movie/Cape-Fear#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/The-Silence-of-the-Lambs#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Prince-of-Tides#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Fisher-King#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Thelma-&-Louise#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/Rambling-Rose#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/For-the-Boys#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Boyz-N-the-Hood#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/JFK#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/Beauty-and-the-Beast#tab=box-office ---

--- Scraping 12/14: https://www.the-numbers.com/movie/Europa-Europa#tab=box-office ---

--- Scraping 13/14: http

# URL 92

In [25]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_92)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Chaplin#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/Unforgiven#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Scent-of-a-Woman#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/The-Crying-Game#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Malcolm-X#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Indochine#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Passion-Fish#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Love-Field#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Lorenzo's-Oil#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/Howards-End#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/The-Player#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/A-Few-Good-Men#tab=box-office ---

--- Scraping 13/15: https://www.the-numbers.com/movie

# URL 93

In [26]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_93)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/14: https://www.the-numbers.com/movie/In-the-Name-of-the-Father#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/14: https://www.the-numbers.com/movie/What's-Love-Got-to-Do-with-It#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/Philadelphia#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Remains-of-the-Day#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/Schindler's-List#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Six-Degrees-of-Separation#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Piano#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/Shadowlands#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Short-Cuts#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/The-Fugitive#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/The-Age-of-Innocence#tab=box-office ---

--- Scraping 12/14: https://www.the-numbers.com/movie/Dave#tab=box-office ---

--- Scraping

# URL 94

In [27]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_94)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/The-Shawshank-Redemption#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/Forrest-Gump#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/The-Madness-of-King-George#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Nobody's-Fool#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Pulp-Fiction#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Nell#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Blue-Sky#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Tom-&-Viv#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Client#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/Bullets-over-Broadway#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/Quiz-Show#tab=box-office ---

--- Scraping 13/15: https://www.the-numbers.co

# URL 95

In [28]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_95)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/14: https://www.the-numbers.com/movie/Leaving-Las-Vegas#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/14: https://www.the-numbers.com/movie/Mr.-Holland's-Opus#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/Nixon#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/Dead-Man-Walking#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Postman-(Il-Postino)#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Casino#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Bridges-of-Madison-County#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/Sense-and-Sensibility#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Babe#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/Braveheart#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/Apollo-13#tab=box-office ---

--- Scraping 12/14: https://www.the-numbers.com/movie/Mighty-Aphrodite#tab=box-office ---

--- Scraping 13/14: http

# URL 96

In [29]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_96)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/13: https://www.the-numbers.com/movie/Jerry-Maguire#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/13: https://www.the-numbers.com/movie/The-English-Patient#tab=box-office ---

--- Scraping 3/13: https://www.the-numbers.com/movie/The-People-vs.-Larry-Flynt#tab=box-office ---

--- Scraping 4/13: https://www.the-numbers.com/movie/Shine#tab=box-office ---

--- Scraping 5/13: https://www.the-numbers.com/movie/Sling-Blade#tab=box-office ---

--- Scraping 6/13: https://www.the-numbers.com/movie/Secrets-&-Lies#tab=box-office ---

--- Scraping 7/13: https://www.the-numbers.com/movie/Marvin's-Room#tab=box-office ---

--- Scraping 8/13: https://www.the-numbers.com/movie/Fargo#tab=box-office ---

--- Scraping 9/13: https://www.the-numbers.com/movie/Breaking-the-Waves#tab=box-office ---

--- Scraping 10/13: https://www.the-numbers.com/movie/The-Crucible#tab=box-office ---

--- Scraping 11/13: https://www.the-numbers.com/movie/Hamlet#tab=box-office ---

--- Scraping 12/13: https://www.the-numbers.com/movie/Trainspotting#tab=box-office ---

--- Scraping 13/13: https://www.the-numb

# URL 97

In [30]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_97)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Good-Will-Hunting#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/The-Apostle#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Ulee's-Gold#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Wag-the-Dog#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/As-Good-as-It-Gets#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/The-Wings-of-the-Dove#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Afterglow#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Mrs.-Brown#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Titanic#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Full-Monty#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/L.A.-Confidential#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/The-Sweet-Hereafter#tab=box-office ---

--- Scraping 13/15: https://www.th

# URL 2015

In [31]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_15)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/17: https://www.the-numbers.com/movie/Trumbo#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/17: https://www.the-numbers.com/movie/The-Martian#tab=box-office ---

--- Scraping 3/17: https://www.the-numbers.com/movie/The-Revenant#tab=box-office ---

--- Scraping 4/17: https://www.the-numbers.com/movie/Steve-Jobs#tab=box-office ---

--- Scraping 5/17: https://www.the-numbers.com/movie/The-Danish-Girl#tab=box-office ---

--- Scraping 6/17: https://www.the-numbers.com/movie/Carol#tab=box-office ---

--- Scraping 7/17: https://www.the-numbers.com/movie/Room#tab=box-office ---

--- Scraping 8/17: https://www.the-numbers.com/movie/Joy#tab=box-office ---

--- Scraping 9/17: https://www.the-numbers.com/movie/45-Years#tab=box-office ---

--- Scraping 10/17: https://www.the-numbers.com/movie/Brooklyn#tab=box-office ---

--- Scraping 11/17: https://www.the-numbers.com/movie/The-Big-Short#tab=box-office ---

--- Scraping 12/17: https://www.the-numbers.com/movie/Mad-Max:-Fury-Road#tab=box-office ---

--- Scraping 13/17: https://www.the-numbers.com/movie/Spotlight#tab=box-off

# URL 2016

In [32]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_16)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/16: https://www.the-numbers.com/movie/Manchester-by-the-Sea#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/16: https://www.the-numbers.com/movie/Hacksaw-Ridge#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/La-La-Land#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Captain-Fantastic#tab=box-office ---

--- Scraping 5/16: https://www.the-numbers.com/movie/Fences#tab=box-office ---

--- Scraping 6/16: https://www.the-numbers.com/movie/Elle#tab=box-office ---

--- Scraping 7/16: https://www.the-numbers.com/movie/Loving#tab=box-office ---

--- Scraping 8/16: https://www.the-numbers.com/movie/Jackie#tab=box-office ---

--- Scraping 9/16: https://www.the-numbers.com/movie/Florence-Foster-Jenkins#tab=box-office ---

--- Scraping 10/16: https://www.the-numbers.com/movie/Arrival#tab=box-office ---

--- Scraping 11/16: https://www.the-numbers.com/movie/Moonlight#tab=box-office ---

--- Scraping 12/16: https://www.the-numbers.com/movie/Hell-or-High-Water#tab=box-office ---

--- Scraping 13/16: https://www.the-numbers.com/movie/Hidden-Fi

# URL 2017

In [33]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_17)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/16: https://www.the-numbers.com/movie/Call-Me-by-Your-Name#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/16: https://www.the-numbers.com/movie/Phantom-Thread#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/Get-Out#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Darkest-Hour#tab=box-office ---

--- Scraping 5/16: https://www.the-numbers.com/movie/Roman-J.-Israel,-Esq.#tab=box-office ---

--- Scraping 6/16: https://www.the-numbers.com/movie/The-Shape-of-Water#tab=box-office ---

--- Scraping 7/16: https://www.the-numbers.com/movie/Three-Billboards-outside-Ebbing,-Missouri#tab=box-office ---

--- Scraping 8/16: https://www.the-numbers.com/movie/I,-Tonya#tab=box-office ---

--- Scraping 9/16: https://www.the-numbers.com/movie/Lady-Bird#tab=box-office ---

--- Scraping 10/16: https://www.the-numbers.com/movie/The-Post#tab=box-office ---

--- Scraping 11/16: https://www.the-numbers.com/movie/Dunkirk#tab=box-office ---

--- Scraping 12/16: https://www.the-numbers.com/movie/The-Disaster-Artist#tab=box-office ---

--- Scraping 13/16

# URL 2018

In [35]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_18)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Vice#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/A-Star-Is-Born#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/At-Eternity's-Gate#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Bohemian-Rhapsody#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Green-Book#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Roma#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/The-Wife#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/The-Favourite#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Can-You-Ever-Forgive-Me?#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/BlacKkKlansman#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/Cold-War#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/Black-Panther#tab=box-office ---

--- Scraping 13/15: https://www.the-num

# URL 2019

In [39]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_19)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Pain-and-Glory#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Once-upon-a-Time...in-Hollywood#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Marriage-Story#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 4/15: https://www.the-numbers.com/movie/Joker#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/The-Two-Popes#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Harriet#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Bombshell#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Judy#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Irishman#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/1917#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/Parasite#tab=box-office ---

--- Scraping 13/15: https://www.the-numbers.com/movie/Ford-v-Ferrari#tab=box-office ---

--- Scraping 14/15: https://www.the-numbers.com/movie/Jojo-Rabbit#tab=box-office ---

--- Scraping 15/15: https://www.the-numbers.com/movie/Knives-Out#tab=box-office -

# URL 2020

In [37]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_18)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Vice#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/A-Star-Is-Born#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/At-Eternity's-Gate#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Bohemian-Rhapsody#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Green-Book#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Roma#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/The-Wife#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/The-Favourite#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Can-You-Ever-Forgive-Me?#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/BlacKkKlansman#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/Cold-War#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/Black-Panther#tab=box-office ---

--- Scraping 13/15: https://www.the-num

# URL 2021

In [38]:
# create scraper
scraper = BoxOfficeNumbersScraper(url_18)

# run scraping
results = scraper.scrape_all()

# domestic summary
summary = scraper.summary_domestic(results)

print(summary)
summary.to_csv("box_office_numbers_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Vice#tab=box-office ---


/var/folders/ls/m7404d6s3_d33l67nvzgd96m0000gn/T/ipykernel_14671/141218213.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements = soup.find_all(text=re.compile(r"Domestic Box Office", re.I))



--- Scraping 2/15: https://www.the-numbers.com/movie/A-Star-Is-Born#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/At-Eternity's-Gate#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Bohemian-Rhapsody#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Green-Book#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Roma#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/The-Wife#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/The-Favourite#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Can-You-Ever-Forgive-Me?#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/BlacKkKlansman#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/Cold-War#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/Black-Panther#tab=box-office ---

--- Scraping 13/15: https://www.the-num